In this example we are going to scrape [Massachusett's Office of Health and Human Services (EOHHS) License Verification Site](https://checkahealthlicense.mass.gov/) for optometrists.

Traditionally Python programmers use [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) to scrape content from the interent. Instead of being *traditional*, we're going to use [Playwright](https://playwright.dev/python/), a **browser automation tool**! This means you actually control the browser! Filling out forms, clicking buttons, downloading documents... it's magic!!!✨✨✨


# Massachusetts Optometrists

- Clicking
- Downloading files

## Installation

We need to install a few tools first! Remove the `#` and run the cell to install the Python packages and browsers that we'll need for our scraping adventure.

In [ ]:
# %pip install --quiet lxml html5lib beautifulsoup4 pandas
# %pip install --quiet playwright
# !playwright install

## Opening up the browser and visiting our destination


In [ ]:
from playwright.async_api import async_playwright

# "Hey, open up a browser"
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

# Create a new browser window
page = await browser.new_page()

In [ ]:
await page.goto("https://checkahealthlicense.mass.gov/")

## Selecting an option from a dropdown

You always start with `await page.locator("select").select_option("whatever option you want")`. You'll probably get an error because there are multiple dropdowns on the page, but Playwright doesn't know which one you want to use! Just read the error and figure out the right one.

In [ ]:
# await page.locator("select").select_option("Optometry")
await page.get_by_label("License Board *").select_option("Board of Registration in Optometry")

In [ ]:
# await page.get_by_text("Search").click()
await page.get_by_label("Click here to search profiles").click()

## Click the download button

This page is great because *there's a download button!* You can't just `.click()` it, though. You need to tell the browser it's about to download something, then make sure it downloads it with the correct filename.

In [ ]:
await page.get_by_label("Click here to open up the options menu for search results below.").click()

You *would* use this code to just click it, but like I said: you can't!

In [ ]:
# await page.get_by_text("Export Report to CSV File").click()

In [ ]:
# Start waiting for the download
async with page.expect_download() as download_info:
    # Perform the action that initiates download
    await page.get_by_text("Export Report to CSV File").click()
download = await download_info.value

print("Saving as", download.suggested_filename)
# Wait for the download process to complete and save the downloaded file somewhere
await download.save_as(download.suggested_filename)